<a href="https://colab.research.google.com/github/TienLungSun/LangChain/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://python.langchain.com/v0.2/docs/tutorials/rag/

In [1]:
pip install langchain langchain_community langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 802.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s 

In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [3]:
pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.6 MB/s eta 0:00:00


In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

··········


把pdf上傳到Colab

In [10]:
for filename in os.listdir():
  if filename.endswith(".pdf"):
    print(filename)

2024 上活動規劃.pdf


In [34]:
from langchain_community.document_loaders import PyPDFLoader

In [44]:
all_docs = []
for filename in os.listdir():
  if filename.endswith(".pdf"):
    loader = PyPDFLoader(filename)
    docs = loader.load()
    for i in range(len(docs)):
      print(docs[i].metadata, end = ",")
    print()
    all_docs.extend(docs)

{'source': '2023活動規劃.pdf', 'page': 0},{'source': '2023活動規劃.pdf', 'page': 1},
{'source': '2024 上活動規劃.pdf', 'page': 0},{'source': '2024 上活動規劃.pdf', 'page': 1},


In [45]:
len(all_docs)

4

In [46]:
all_docs[0].metadata

{'source': '2023活動規劃.pdf', 'page': 0}

In [47]:
all_docs[0].page_content[0:100]

'半導體書院「人際素養」活動規劃  \n結合「企業導師」與學校「書院導師」雙軌制度，培養學生未來職場之人際互動素養。  \nWeek  Date  地點 活動類別 人數 時數 主題/演講題目/ 講者  備註'

In [55]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, add_start_index=True
)

all_splits = text_splitter.split_documents(all_docs)
print(len(all_splits))

7


In [56]:
for i in range(len(all_splits)):
  print(all_splits[i].metadata)

{'source': '2023活動規劃.pdf', 'page': 0, 'start_index': 0}
{'source': '2023活動規劃.pdf', 'page': 0, 'start_index': 392}
{'source': '2023活動規劃.pdf', 'page': 1, 'start_index': 0}
{'source': '2024 上活動規劃.pdf', 'page': 0, 'start_index': 0}
{'source': '2024 上活動規劃.pdf', 'page': 0, 'start_index': 393}
{'source': '2024 上活動規劃.pdf', 'page': 0, 'start_index': 801}
{'source': '2024 上活動規劃.pdf', 'page': 1, 'start_index': 0}


In [67]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [62]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("AI 影像辨識應用實務")
print(len(retrieved_docs))

6


In [65]:
for i in range(len(retrieved_docs)):
  print(retrieved_docs[i].metadata)

{'page': 0, 'source': '2023活動規劃.pdf', 'start_index': 392}
{'page': 0, 'source': '2023活動規劃.pdf', 'start_index': 0}
{'page': 0, 'source': '2023活動規劃.pdf', 'start_index': 0}
{'page': 0, 'source': '2024 上活動規劃.pdf', 'start_index': 393}
{'page': 1, 'source': '2024 上活動規劃.pdf', 'start_index': 0}
{'page': 1, 'source': '2024 上活動規劃.pdf', 'start_index': 0}


In [66]:
retrieved_docs = retriever.invoke("半導體")
for i in range(len(retrieved_docs)):
  print(retrieved_docs[i].metadata)

{'page': 0, 'source': '2024 上活動規劃.pdf', 'start_index': 393}
{'page': 0, 'source': '2023活動規劃.pdf', 'start_index': 0}
{'page': 0, 'source': '2023活動規劃.pdf', 'start_index': 0}
{'page': 0, 'source': '2024 上活動規劃.pdf', 'start_index': 0}
{'page': 0, 'source': '2024 上活動規劃.pdf', 'start_index': 0}
{'page': 1, 'source': '2023活動規劃.pdf', 'start_index': 0}


In [31]:
pip install -qU langchain-openai

In [32]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

··········


In [70]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [71]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [73]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("10/4 的講者是誰？"):
    print(chunk, end="", flush=True)

10/4 的講者是台積電智能製造中心智能製程控制系統部的林晏經理。

In [74]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
)

In [75]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "誰演講ＡＩ"})
print(response["answer"])

在2023年10月18日的演講中，旺宏電子的謝光宇處長主講了「AI的興起與半導體技術的變革」。


In [78]:
for document in response["context"]:
    print(document.metadata)
    print()

{'page': 1, 'source': '2023活動規劃.pdf', 'start_index': 0}

{'page': 1, 'source': '2023活動規劃.pdf', 'start_index': 0}

{'page': 1, 'source': '2023活動規劃.pdf', 'start_index': 0}

{'page': 0, 'source': '2023活動規劃.pdf', 'start_index': 0}

{'page': 0, 'source': '2023活動規劃.pdf', 'start_index': 0}

{'page': 0, 'source': '2023活動規劃.pdf', 'start_index': 0}



In [79]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("誰演講ESG")

'演講ESG的是職安衛暨風險管理部的吳鴻明副處長。謝謝提問！'